In [10]:
## Import dependencies 

import pandas as pd

In [11]:
## Read CSV file into Pandas DataFrame from URL 
## The target URL is the ASX company directory, this data will be scraped and stored in a PostgreSQL database 
## Creating a web-scraping programme will enable the database to be updated automatically when new companies are added to the index 

# Create URL object, assigning the target URL 

URL = "https://asx.api.markitdigital.com/asx-research/1.0/companies/directory/file?access_token=83ff96335c2d45a094df02a206a39ff4"

# Create data object to store results, use pd.read_csv to return DataFrame 

data = pd.read_csv(URL)    

# Print results 

data

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D,1414 DEGREES LIMITED,Capital Goods,12/09/2018,9452331
1,1AD,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,8800309
2,1AE,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,9804506
3,1AG,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,6965525
4,1CG,ONE CLICK GROUP LIMITED,Capital Goods,28/04/2017,4801323
...,...,...,...,...,...
2053,ZLD,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,17134204
2054,ZMI,ZINC OF IRELAND NL,Materials,18/09/2007,4476030
2055,ZMM,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,3344847
2056,ZNC,ZENITH MINERALS LIMITED,Materials,29/05/2007,30657137


In [12]:
## Import dependencies 
# yahooquery library will be used to access quotes 

from yahooquery import Ticker

In [13]:
# Add the .AX suffix to the ASX code, this is required to exectue the yahooquery search

data['ASX code'] = data['ASX code'].astype(str) + '.AX'  

# Print results 

data

,ASX code,Company name,GICs industry group,Listing date,Market Cap
0,14D.AX,1414 DEGREES LIMITED,Capital Goods,12/09/2018,9452331
1,1AD.AX,ADALTA LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",22/08/2016,8800309
2,1AE.AX,AURORA ENERGY METALS LIMITED,Materials,18/05/2022,9804506
3,1AG.AX,ALTERRA LIMITED,"Food, Beverage & Tobacco",16/05/2008,6965525
4,1CG.AX,ONE CLICK GROUP LIMITED,Capital Goods,28/04/2017,4801323
...,...,...,...,...,...
2053,ZLD.AX,ZELIRA THERAPEUTICS LIMITED,"Pharmaceuticals, Biotechnology & Life Sciences",28/07/2003,17134204
2054,ZMI.AX,ZINC OF IRELAND NL,Materials,18/09/2007,4476030
2055,ZMM.AX,ZIMI LIMITED,Technology Hardware & Equipment,10/09/2007,3344847
2056,ZNC.AX,ZENITH MINERALS LIMITED,Materials,29/05/2007,30657137


In [24]:
## Create Ticker object with each ticker in the ASX code column 

ticker = Ticker(data['ASX code'], asynchronous=True)

# Request price history for each ticker, over a period of 1 year 

stockPrice = ticker.history(period="1y").reset_index()

# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,adjclose
0,14D.AX,2022-07-04,0.079,0.079,0.076,0.0790,39690.0,0.0790
1,14D.AX,2022-07-05,0.079,0.079,0.077,0.0790,37384.0,0.0790
2,14D.AX,2022-07-06,0.077,0.079,0.077,0.0790,695.0,0.0790
3,14D.AX,2022-07-07,0.079,0.079,0.079,0.0790,0.0,0.0790
4,14D.AX,2022-07-08,0.080,0.085,0.080,0.0835,22216.0,0.0835
...,...,...,...,...,...,...,...,...
502232,ZNO.AX,2023-06-26,0.041,0.042,0.040,0.0420,311281.0,0.0420
502233,ZNO.AX,2023-06-27,0.040,0.040,0.035,0.0380,282082.0,0.0380
502234,ZNO.AX,2023-06-28,0.036,0.036,0.036,0.0360,163213.0,0.0360
502235,ZNO.AX,2023-06-29,0.034,0.035,0.033,0.0340,106928.0,0.0340


In [25]:
## Delete the DataFrame columns that are not required

stockPrice.drop(['adjclose'], axis=1, inplace=True)

In [26]:
## Calculate the percentage change, based on closing price of the market, by using the pct_change method

stockPrice['percentage change'] = (stockPrice['close'] - stockPrice['open']) / stockPrice['open']

# Round the numerical data to three decimal places 

stockPrice = stockPrice.round(3)

# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,percentage change
0,14D.AX,2022-07-04,0.079,0.079,0.076,0.079,39690.0,0.000
1,14D.AX,2022-07-05,0.079,0.079,0.077,0.079,37384.0,0.000
2,14D.AX,2022-07-06,0.077,0.079,0.077,0.079,695.0,0.026
3,14D.AX,2022-07-07,0.079,0.079,0.079,0.079,0.0,0.000
4,14D.AX,2022-07-08,0.080,0.085,0.080,0.083,22216.0,0.044
...,...,...,...,...,...,...,...,...
502232,ZNO.AX,2023-06-26,0.041,0.042,0.040,0.042,311281.0,0.024
502233,ZNO.AX,2023-06-27,0.040,0.040,0.035,0.038,282082.0,-0.050
502234,ZNO.AX,2023-06-28,0.036,0.036,0.036,0.036,163213.0,0.000
502235,ZNO.AX,2023-06-29,0.034,0.035,0.033,0.034,106928.0,0.000


In [27]:
# Create function to assign gain, loss or no change to stock price movement based on percentage change value

def f(stockPrice):
    if stockPrice['percentage change'] < 0:
        val = "Loss"
    elif stockPrice['percentage change'] > 0:
        val = "Gain"
    else:
        val = "No Change"
    return val

In [28]:
# Create new column titled gain/loss and apply function created above to assign appropriate values 

stockPrice['gain/loss'] = stockPrice.apply(f, axis=1)

In [29]:
# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,percentage change,gain/loss
0,14D.AX,2022-07-04,0.079,0.079,0.076,0.079,39690.0,0.000,No Change
1,14D.AX,2022-07-05,0.079,0.079,0.077,0.079,37384.0,0.000,No Change
2,14D.AX,2022-07-06,0.077,0.079,0.077,0.079,695.0,0.026,Gain
3,14D.AX,2022-07-07,0.079,0.079,0.079,0.079,0.0,0.000,No Change
4,14D.AX,2022-07-08,0.080,0.085,0.080,0.083,22216.0,0.044,Gain
...,...,...,...,...,...,...,...,...,...
502232,ZNO.AX,2023-06-26,0.041,0.042,0.040,0.042,311281.0,0.024,Gain
502233,ZNO.AX,2023-06-27,0.040,0.040,0.035,0.038,282082.0,-0.050,Loss
502234,ZNO.AX,2023-06-28,0.036,0.036,0.036,0.036,163213.0,0.000,No Change
502235,ZNO.AX,2023-06-29,0.034,0.035,0.033,0.034,106928.0,0.000,No Change


In [37]:
## Create columns to store simple moving average (SMA) calculations
## The 20 day, 50 day and 100 intervals have been selected 

# The rolling.mean method is used, with the window setting the interval for the moving average

# SMA 20

stockPrice['SMA 20'] = stockPrice['close'].rolling(window=20).mean()

# SMA 50 

stockPrice['SMA 50'] = stockPrice['close'].rolling(window=50).mean()

# SMA 100
stockPrice['SMA 100'] = stockPrice['close'].rolling(window=100).mean()

In [38]:
## Calculate 20 day SMA for volume 

stockPrice['Volume SMA 20'] = stockPrice['volume'].rolling(window=20).mean()

In [39]:
# Print results 

stockPrice

,symbol,date,open,high,low,close,volume,percentage change,gain/loss,SMA 20,SMA 50,SMA 100,Volume SMA 20
0,14D.AX,2022-07-04,0.079,0.079,0.076,0.079,39690.0,0.000,No Change,NaN,NaN,NaN,NaN
1,14D.AX,2022-07-05,0.079,0.079,0.077,0.079,37384.0,0.000,No Change,NaN,NaN,NaN,NaN
2,14D.AX,2022-07-06,0.077,0.079,0.077,0.079,695.0,0.026,Gain,NaN,NaN,NaN,NaN
3,14D.AX,2022-07-07,0.079,0.079,0.079,0.079,0.0,0.000,No Change,NaN,NaN,NaN,NaN
4,14D.AX,2022-07-08,0.080,0.085,0.080,0.083,22216.0,0.044,Gain,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
502232,ZNO.AX,2023-06-26,0.041,0.042,0.040,0.042,311281.0,0.024,Gain,0.04485,0.04854,0.06142,88631.35
502233,ZNO.AX,2023-06-27,0.040,0.040,0.035,0.038,282082.0,-0.050,Loss,0.04450,0.04820,0.06085,102360.65
502234,ZNO.AX,2023-06-28,0.036,0.036,0.036,0.036,163213.0,0.000,No Change,0.04380,0.04780,0.06028,108627.80
502235,ZNO.AX,2023-06-29,0.034,0.035,0.033,0.034,106928.0,0.000,No Change,0.04320,0.04740,0.05972,113682.70
